# ***Demo: Single-GPU Training with Amazon SageMaker***

In [1]:
# !pip install sagemaker --upgrade -q
# !pip install ipywidgets -q
# !wget https://raw.githubusercontent.com/awslabs/amazon-sagemaker-examples/master/advanced_functionality/tensorflow_bring_your_own/utils/generate_cifar10_tfrecords.py
# !python generate_cifar10_tfrecords.py --data-dir cifar10

**Step 1:** Import essentials packages, start a sagemaker session and specify the bucket name you created in the pre-requsites section of this workshop.

In [2]:
import os
import boto3
import time
import numpy as np
import sagemaker

sess = boto3.Session()
sm   = sess.client('sagemaker')
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

bucket_name    = sagemaker_session.default_bucket()
jobs_folder    = 'jobs'
dataset_folder = 'datasets'

![](https://miro.medium.com/max/1000/0*GRfvsrvtfpRm400-)

#### Upload dataset if it doesn't exist

In [3]:
# datasets = sagemaker_session.upload_data(path='cifar10', key_prefix=f'{dataset_folder}/cifar10-dataset')
# datasets

#If dataset already exists
datasets = f's3://{bucket_name}/{dataset_folder}/cifar10-dataset'
datasets

's3://sagemaker-us-west-2-453691756499/datasets/cifar10-dataset'

**Step 2:** Specify hyperparameters, instance type and number of instances to distribute training to. 

In [4]:
job_name   = f'tf-single-gpu-{time.strftime("%Y-%m-%d-%H-%M-%S-%j", time.gmtime())}'
output_path = f's3://{bucket_name}/{jobs_folder}'

metric_definitions = [{'Name': 'Validation Accuracy', 'Regex': 'Validation Accuracy: ([0-9\\.]+)'}]

hyperparameters = {'epochs'       : 50, 
                   'learning-rate': 0.01,
                   'momentum'     : 0.95,
                   'weight-decay' : 2e-4,
                   'optimizer'    : 'adam',
                   'batch-size'   : 256,
                   'model-type'   : 'custom'}


In [5]:
from sagemaker.debugger import TensorBoardOutputConfig

tensorboard_output_config = TensorBoardOutputConfig(
    s3_output_path=f's3://{bucket_name}/tensorboard_logs')

In [6]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(entry_point         = 'cifar10-tf2.py', 
                           source_dir           = 'code',
                           output_path          = output_path + '/',
                           code_location        = output_path,
                           role                 = role,
                           instance_count       = 1,
                           instance_type        = 'ml.p3.2xlarge',
                           framework_version    = '2.4', 
                           py_version           = 'py37',
                           metric_definitions   = metric_definitions,
                           hyperparameters      = hyperparameters,
                           tensorboard_output_config = tensorboard_output_config)

**Step 4:** Specify dataset locations in Amazon S3 and then call the fit function.

In [7]:
estimator.fit({'train': datasets,
                    'validation': datasets,
                    'eval': datasets}, 
                  job_name=job_name, wait=True)

2021-04-17 23:11:56 Starting - Starting the training job...
2021-04-17 23:12:20 Starting - Launching requested ML instancesProfilerReport-1618701116: InProgress
......
2021-04-17 23:13:20 Starting - Preparing the instances for training.........
2021-04-17 23:14:52 Downloading - Downloading input data
2021-04-17 23:14:52 Training - Downloading the training image..........2021-04-17 23:16:31.579638: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-04-17 23:16:31.584154: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2021-04-17 23:16:31.661692: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-04-17 23:16:31.745669: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-04-